In [1]:
import re
from pathlib import Path
import pandas as pd

# ====== CONFIG ======
DEFAULT_PATTERN = "Serial Bluetooth Terminal *.txt"
OUT_XLSX = "resumen_sesiones_rodalert_TFM.xlsx"
OUT_REPORT_MD = "resumen_TFM_rodalert.md"
OUT_FIG_DIR = "figuras_rodalert"

# ====== Helpers ======
def strip_timestamp(line: str) -> str:
    return re.sub(r"^\d{2}:\d{2}:\d{2}\s+", "", line).strip()

def parse_date_from_filename(filename: str):
    m = re.search(r"(\d{8})-\d{6}", filename)
    if not m:
        return None
    yyyymmdd = m.group(1)
    return f"{yyyymmdd[0:4]}-{yyyymmdd[4:6]}-{yyyymmdd[6:8]}"

def extract_session_metadata(lines, resumen_idx):
    paciente_id = None
    condicion = None
    for k in range(resumen_idx, -1, -1):
        s = strip_timestamp(lines[k])

        if paciente_id is None:
            m_id = re.search(r"^-{10,}\s*(.+)$", s)  # tolerante: 10+ guiones
            if m_id:
                paciente_id = m_id.group(1).strip()

        if condicion is None:
            m_cond = re.search(r"^Condici[oó]n:\s*(.+)$", s, flags=re.IGNORECASE)
            if m_cond:
                condicion = m_cond.group(1).strip()

        if paciente_id and condicion:
            break
    return paciente_id, condicion

def parse_summary_block(block_text: str) -> dict:
    lines = [strip_timestamp(l) for l in block_text.splitlines()]
    text = "\n".join(lines)

    m = re.search(r"Duraci[oó]n:\s*(\d+)\s*s", text, flags=re.IGNORECASE)
    duracion_s = int(m.group(1)) if m else None

    m = re.search(r"Distancia minima MINf:\s*([0-9]+(?:\.[0-9]+)?)\s*cm", text, flags=re.IGNORECASE)
    minf_cm = float(m.group(1)) if m else None

    m = re.search(r"Choques totales:\s*(\d+)", text, flags=re.IGNORECASE)
    choques_tot = int(m.group(1)) if m else 0
    choques_global_times = re.findall(r"Choque_global #\d+\s+en t=(\d+)\s*s", text)

    # SOLO S1/S2/S4 (S3 NO choques)
    def parse_sensor(sensor_label, sensor_tag):
        m = re.search(rf"{re.escape(sensor_label)}:\s*(\d+)", text, flags=re.IGNORECASE)
        n = int(m.group(1)) if m else 0
        times = re.findall(rf"Choque_{sensor_tag}\s+#\d+\s+en t=(\d+)\s*s", text)
        return n, ", ".join(times)

    choq_s1, t_s1 = parse_sensor("S1 frontal", "S1")
    choq_s2, t_s2 = parse_sensor("S2 frontal", "S2")
    choq_s4, t_s4 = parse_sensor("S4 lateral", "S4")

    m = re.search(r"Activaciones buzzer:\s*(\d+)", text, flags=re.IGNORECASE)
    buz_n = int(m.group(1)) if m else 0
    buz_t = ", ".join(re.findall(r"Buzzer #\d+\s+en t=(\d+)\s*s", text))

    m = re.search(r"Activaciones motor:\s*(\d+)", text, flags=re.IGNORECASE)
    mot_n = int(m.group(1)) if m else 0
    mot_t = ", ".join(re.findall(r"Motor #\d+\s+en t=(\d+)\s*s", text))

    m = re.search(r"Giros S3\s*\(cabeza izq\):\s*(\d+)", text, flags=re.IGNORECASE)
    giros_n = int(m.group(1)) if m else 0
    giros = re.findall(r"Giro_S3 #\d+\s+de t=([\d\.]+)\s*s a t=([\d\.]+)\s*s", text)
    giros_intervals = ", ".join([f"{a}-{b}" for a, b in giros])

    m = re.search(r"falsos negativos\):\s*(\d+)", text, flags=re.IGNORECASE)
    fn = int(m.group(1)) if m else None
    m = re.search(r"falsos positivos\):\s*(\d+)", text, flags=re.IGNORECASE)
    fp = int(m.group(1)) if m else None
    m = re.search(r"Tiempo medio aviso->choque:\s*([0-9]+(?:\.[0-9]+)?)\s*s", text, flags=re.IGNORECASE)
    aviso_choque_media_s = float(m.group(1)) if m else None

    return {
        "Duracion_s": duracion_s,
        "MINf_cm": minf_cm,

        "Choques_totales": choques_tot,
        "Choques_global_tiempos_s": ", ".join(choques_global_times),

        "Choques_S1": choq_s1,
        "Choques_S1_tiempos_s": t_s1,
        "Choques_S2": choq_s2,
        "Choques_S2_tiempos_s": t_s2,
        "Choques_S4": choq_s4,
        "Choques_S4_tiempos_s": t_s4,

        "Buzzer_activaciones": buz_n,
        "Buzzer_tiempos_s": buz_t,

        "Motor_activaciones": mot_n,
        "Motor_tiempos_s": mot_t,

        "Giros_S3_totales": giros_n,
        "Giros_S3_intervalos_s": giros_intervals,

        "FN_choque_sin_aviso_3s": fn,
        "FP_aviso_sin_choque_5s": fp,
        "Aviso_a_choque_media_s": aviso_choque_media_s,
    }

def parse_txt_file(path: Path):
    text = path.read_text(encoding="utf-8", errors="ignore")
    lines = text.splitlines()

    results = []
    sesion_idx = 0

    for i, line in enumerate(lines):
        if "---- RESUMEN DE LA SESION ----" in line:
            sesion_idx += 1
            block = [line]
            j = i + 1
            while j < len(lines):
                if "NUEVA SESION?" in lines[j] or "---- RESUMEN DE LA SESION ----" in lines[j]:
                    break
                block.append(lines[j])
                j += 1

            data = parse_summary_block("\n".join(block))
            paciente_id, condicion = extract_session_metadata(lines, i)

            data["Paciente_ID"] = paciente_id
            data["Condicion"] = condicion
            data["Dia"] = parse_date_from_filename(path.name)
            data["Fichero"] = path.name
            data["Sesion_idx_en_fichero"] = sesion_idx

            # tasas por minuto
            dur = data.get("Duracion_s") or 0
            mins = dur / 60.0 if dur else None
            for col in ["Choques_totales", "Buzzer_activaciones", "Motor_activaciones", "Giros_S3_totales"]:
                data[col + "_por_min"] = (data.get(col, 0) or 0) / mins if (mins and mins > 0) else None

            results.append(data)

    return results

# ====== Excel formatting ======
def autosize_and_wrap_excel(path_xlsx: str, sheet_names=None):
    from openpyxl import load_workbook
    from openpyxl.styles import Alignment

    wb = load_workbook(path_xlsx)
    if sheet_names is None:
        sheet_names = wb.sheetnames

    for sh in sheet_names:
        ws = wb[sh]
        ws.freeze_panes = "A2"

        wrap = Alignment(wrap_text=True, vertical="top")
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.alignment = wrap

        for col_cells in ws.columns:
            max_len = 0
            col_letter = col_cells[0].column_letter
            for cell in col_cells:
                val = "" if cell.value is None else str(cell.value)
                if len(val) > max_len:
                    max_len = len(val)
            ws.column_dimensions[col_letter].width = min(max_len + 2, 60)

    wb.save(path_xlsx)

# ====== Plots ======
def make_plots(df: pd.DataFrame, out_dir: str = OUT_FIG_DIR):
    import matplotlib.pyplot as plt

    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # --- 1) Por día (líneas/barras) ---
    if "Dia" in df.columns:
        by_day = (df.groupby("Dia", dropna=False)
                    .agg({
                        "Choques_totales_por_min": "mean",
                        "Buzzer_activaciones_por_min": "mean",
                        "Motor_activaciones_por_min": "mean",
                        "Giros_S3_totales_por_min": "mean",
                        "MINf_cm": "min",
                        "Duracion_s": "sum"
                    })
                    .reset_index())

        # Choques/min por día
        plt.figure()
        plt.plot(by_day["Dia"], by_day["Choques_totales_por_min"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("Choques por minuto (media del día)")
        plt.xlabel("Día")
        plt.title("Choques/min por día")
        plt.tight_layout()
        plt.savefig(Path(out_dir) / "por_dia_choques_por_min.png", dpi=200)
        plt.close()

        # Buzzer/min por día
        plt.figure()
        plt.plot(by_day["Dia"], by_day["Buzzer_activaciones_por_min"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("Buzzer por minuto (media del día)")
        plt.xlabel("Día")
        plt.title("Buzzer/min por día")
        plt.tight_layout()
        plt.savefig(Path(out_dir) / "por_dia_buzzer_por_min.png", dpi=200)
        plt.close()

        # Motor/min por día
        plt.figure()
        plt.plot(by_day["Dia"], by_day["Motor_activaciones_por_min"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("Motor por minuto (media del día)")
        plt.xlabel("Día")
        plt.title("Motor/min por día")
        plt.tight_layout()
        plt.savefig(Path(out_dir) / "por_dia_motor_por_min.png", dpi=200)
        plt.close()

        # Giros/min por día
        plt.figure()
        plt.plot(by_day["Dia"], by_day["Giros_S3_totales_por_min"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("Giros S3 por minuto (media del día)")
        plt.xlabel("Día")
        plt.title("Giros/min por día")
        plt.tight_layout()
        plt.savefig(Path(out_dir) / "por_dia_giros_por_min.png", dpi=200)
        plt.close()

        # MINf por día (mínimo)
        plt.figure()
        plt.plot(by_day["Dia"], by_day["MINf_cm"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("MINf (cm) - mínimo del día")
        plt.xlabel("Día")
        plt.title("Distancia mínima (MINf) por día")
        plt.tight_layout()
        plt.savefig(Path(out_dir) / "por_dia_MINf.png", dpi=200)
        plt.close()

    # --- 2) Boxplots por condición ---
    if "Condicion" in df.columns:
        # orden fijo para que siempre salga igual
        cond_order = ["SIN NADA", "BUZZER SOLO", "BUZZER + MOTOR"]
        # pero en tus logs puede venir como frase distinta -> lo dejamos tal cual
        condiciones = [c for c in df["Condicion"].dropna().unique().tolist()]
        # intenta un orden "humano"
        condiciones_sorted = sorted(condiciones, key=lambda x: cond_order.index(x) if x in cond_order else 999)

        def boxplot_metric(metric, title, fname):
            data = []
            labels = []
            for c in condiciones_sorted:
                s = df.loc[df["Condicion"] == c, metric].dropna()
                if len(s) > 0:
                    data.append(s.values)
                    labels.append(c)
            if len(data) < 1:
                return
            plt.figure()
            plt.boxplot(data, labels=labels, showfliers=False)
            plt.xticks(rotation=25, ha="right")
            plt.ylabel(metric)
            plt.title(title)
            plt.tight_layout()
            plt.savefig(Path(out_dir) / fname, dpi=200)
            plt.close()

        boxplot_metric("Choques_totales_por_min", "Choques/min por condición", "box_cond_choques_por_min.png")
        boxplot_metric("Buzzer_activaciones_por_min", "Buzzer/min por condición", "box_cond_buzzer_por_min.png")
        boxplot_metric("Motor_activaciones_por_min", "Motor/min por condición", "box_cond_motor_por_min.png")
        boxplot_metric("Giros_S3_totales_por_min", "Giros/min por condición", "box_cond_giros_por_min.png")
        boxplot_metric("MINf_cm", "MINf (cm) por condición", "box_cond_MINf.png")

# ====== Report MD ======
def make_tfm_report(df: pd.DataFrame, out_md: str = OUT_REPORT_MD, fig_dir: str = OUT_FIG_DIR):
    # tablas clave
    by_day = (df.groupby("Dia", dropna=False)
                .agg(
                    n_sesiones=("Sesion_idx_en_fichero", "count"),
                    duracion_total_s=("Duracion_s", "sum"),
                    choques_total=("Choques_totales", "sum"),
                    buzzer_total=("Buzzer_activaciones", "sum"),
                    motor_total=("Motor_activaciones", "sum"),
                    giros_total=("Giros_S3_totales", "sum"),
                    choques_min_media=("Choques_totales_por_min", "mean"),
                    buzzer_min_media=("Buzzer_activaciones_por_min", "mean"),
                    motor_min_media=("Motor_activaciones_por_min", "mean"),
                    giros_min_media=("Giros_S3_totales_por_min", "mean"),
                    MINf_min_cm=("MINf_cm", "min"),
                )
                .reset_index())

    by_cond = (df.groupby("Condicion", dropna=False)
                 .agg(
                     n_sesiones=("Sesion_idx_en_fichero", "count"),
                     duracion_total_s=("Duracion_s", "sum"),
                     choques_total=("Choques_totales", "sum"),
                     choques_min_media=("Choques_totales_por_min", "mean"),
                     buzzer_min_media=("Buzzer_activaciones_por_min", "mean"),
                     motor_min_media=("Motor_activaciones_por_min", "mean"),
                     giros_min_media=("Giros_S3_totales_por_min", "mean"),
                     MINf_min_cm=("MINf_cm", "min"),
                     FN_total=("FN_choque_sin_aviso_3s", "sum"),
                     FP_total=("FP_aviso_sin_choque_5s", "sum"),
                     aviso_a_choque_media_s=("Aviso_a_choque_media_s", "mean"),
                 )
                 .reset_index())

    # interpretación automática MUY prudente
    # (no inventa causalidad, solo describe)
    def safe_float(x):
        try:
            return float(x)
        except:
            return None

    lines = []
    lines.append("# Resumen TFM — RodAlert (auto-generado)\n")
    lines.append("## Qué contiene\n")
    lines.append("- Resumen por sesión (choques, buzzer, motor, giros S3, MINf)\n")
    lines.append("- Agregados por día y por condición\n")
    lines.append("- Figuras guardadas en la carpeta de salida\n")

    lines.append("\n## Tabla por día\n")
    lines.append(by_day.to_markdown(index=False))

    lines.append("\n\n## Tabla por condición\n")
    lines.append(by_cond.to_markdown(index=False))

    # Hallazgos simples
    lines.append("\n\n## Observaciones automáticas (descriptivas)\n")
    # mejor/peor condición por choques/min
    if "choques_min_media" in by_cond.columns and len(by_cond) > 1:
        tmp = by_cond.dropna(subset=["choques_min_media"]).copy()
        if len(tmp) >= 1:
            tmp = tmp.sort_values("choques_min_media")
            best = tmp.iloc[0]
            worst = tmp.iloc[-1]
            lines.append(f"- Menor **Choques/min (media)**: **{best['Condicion']}** = {safe_float(best['choques_min_media']):.3f}")
            if best["Condicion"] != worst["Condicion"]:
                lines.append(f"- Mayor **Choques/min (media)**: **{worst['Condicion']}** = {safe_float(worst['choques_min_media']):.3f}")
            lines.append("  - Nota: esto es *descriptivo*; con pocas sesiones no concluye causalidad.")

    # seguridad (FN/FP)
    if "FN_total" in by_cond.columns:
        tmp = by_cond.dropna(subset=["FN_total"]).copy()
        if len(tmp) >= 1:
            tmp = tmp.sort_values("FN_total")
            best = tmp.iloc[0]
            lines.append(f"- Menos **FN (choque sin aviso en 3 s)**: **{best['Condicion']}** = {int(best['FN_total'])}")

    if "FP_total" in by_cond.columns:
        tmp = by_cond.dropna(subset=["FP_total"]).copy()
        if len(tmp) >= 1:
            tmp = tmp.sort_values("FP_total")
            best = tmp.iloc[0]
            lines.append(f"- Menos **FP (aviso sin choque en 5 s)**: **{best['Condicion']}** = {int(best['FP_total'])}")

    lines.append("\n\n## Figuras generadas\n")
    figs = [
        "por_dia_choques_por_min.png",
        "por_dia_buzzer_por_min.png",
        "por_dia_motor_por_min.png",
        "por_dia_giros_por_min.png",
        "por_dia_MINf.png",
        "box_cond_choques_por_min.png",
        "box_cond_buzzer_por_min.png",
        "box_cond_motor_por_min.png",
        "box_cond_giros_por_min.png",
        "box_cond_MINf.png",
    ]
    for f in figs:
        lines.append(f"- `{fig_dir}/{f}`")

    lines.append("\n\n## Cómo lo puedes usar en tu TFM (ideas de análisis)\n")
    lines.append("- **Eficacia por condición**: comparar choques/min y MINf por condición.")
    lines.append("- **Coste de asistencia**: comparar buzzer/min y motor/min (molestia/energía) frente a choques/min.")
    lines.append("- **Seguridad**: FN y FP por condición (trade-off).")
    lines.append("- **Aprendizaje/efecto del día**: tendencia de choques/min por día (habituación o fatiga).")
    lines.append("- **Relación con giros**: si aumentan giros/min y baja choques/min (exploración vs impacto).")

    Path(out_md).write_text("\n".join(lines), encoding="utf-8")

# ====== Main ======
def procesar_carpeta(
    carpeta=".",
    patron=DEFAULT_PATTERN,
    salida_xlsx=OUT_XLSX,
    salida_md=OUT_REPORT_MD,
    figuras_dir=OUT_FIG_DIR,
):
    folder = Path(carpeta)
    all_rows = []
    for f in folder.glob(patron):
        all_rows.extend(parse_txt_file(f))

    if not all_rows:
        print("No se encontraron sesiones en los .txt.")
        return

    df = pd.DataFrame(all_rows)

    # Orden de columnas (Paciente_ID + Condicion primeras)
    first_cols = [
        "Paciente_ID", "Condicion", "Dia",
        "Duracion_s", "MINf_cm",
        "Choques_totales", "Choques_totales_por_min",
        "Choques_S1", "Choques_S1_tiempos_s",
        "Choques_S2", "Choques_S2_tiempos_s",
        "Choques_S4", "Choques_S4_tiempos_s",
        "Buzzer_activaciones", "Buzzer_activaciones_por_min", "Buzzer_tiempos_s",
        "Motor_activaciones", "Motor_activaciones_por_min", "Motor_tiempos_s",
        "Giros_S3_totales", "Giros_S3_totales_por_min", "Giros_S3_intervalos_s",
        "FN_choque_sin_aviso_3s", "FP_aviso_sin_choque_5s", "Aviso_a_choque_media_s",
        "Choques_global_tiempos_s",
        "Fichero", "Sesion_idx_en_fichero",
    ]
    cols = [c for c in first_cols if c in df.columns] + [c for c in df.columns if c not in first_cols]
    df = df[cols]

    # Agregados
    by_day = (df.groupby("Dia", dropna=False)
                .agg(
                    n_sesiones=("Sesion_idx_en_fichero", "count"),
                    duracion_total_s=("Duracion_s", "sum"),
                    choques_total=("Choques_totales", "sum"),
                    buzzer_total=("Buzzer_activaciones", "sum"),
                    motor_total=("Motor_activaciones", "sum"),
                    giros_total=("Giros_S3_totales", "sum"),
                    choques_min_media=("Choques_totales_por_min", "mean"),
                    buzzer_min_media=("Buzzer_activaciones_por_min", "mean"),
                    motor_min_media=("Motor_activaciones_por_min", "mean"),
                    giros_min_media=("Giros_S3_totales_por_min", "mean"),
                    MINf_min_cm=("MINf_cm", "min"),
                )
                .reset_index())

    by_cond = (df.groupby("Condicion", dropna=False)
                 .agg(
                     n_sesiones=("Sesion_idx_en_fichero", "count"),
                     duracion_total_s=("Duracion_s", "sum"),
                     choques_total=("Choques_totales", "sum"),
                     choques_min_media=("Choques_totales_por_min", "mean"),
                     buzzer_min_media=("Buzzer_activaciones_por_min", "mean"),
                     motor_min_media=("Motor_activaciones_por_min", "mean"),
                     giros_min_media=("Giros_S3_totales_por_min", "mean"),
                     MINf_min_cm=("MINf_cm", "min"),
                     FN_total=("FN_choque_sin_aviso_3s", "sum"),
                     FP_total=("FP_aviso_sin_choque_5s", "sum"),
                     aviso_a_choque_media_s=("Aviso_a_choque_media_s", "mean"),
                 )
                 .reset_index())

    # Excel
    with pd.ExcelWriter(salida_xlsx, engine="openpyxl") as w:
        df.to_excel(w, index=False, sheet_name="Resumen")
        by_day.to_excel(w, index=False, sheet_name="Por_dia")
        by_cond.to_excel(w, index=False, sheet_name="Por_condicion")

    autosize_and_wrap_excel(salida_xlsx, sheet_names=["Resumen", "Por_dia", "Por_condicion"])

    # Figuras + Reporte
    make_plots(df, out_dir=figuras_dir)
    make_tfm_report(df, out_md=salida_md, fig_dir=figuras_dir)

    print(f"OK -> Excel: {salida_xlsx}")
    print(f"OK -> Figuras en: {figuras_dir}/")
    print(f"OK -> Reporte TFM: {salida_md}")

if __name__ == "__main__":
    procesar_carpeta(
        carpeta=".",
        patron=DEFAULT_PATTERN,
        salida_xlsx=OUT_XLSX,
        salida_md=OUT_REPORT_MD,
        figuras_dir=OUT_FIG_DIR,
    )


C:\Users\ASUS\AppData\Local\Temp\ipykernel_38296\1226980365.py:275: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, showfliers=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_38296\1226980365.py:275: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, showfliers=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_38296\1226980365.py:275: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, showfliers=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_38296\1226980365.py:275: MatplotlibDeprecationWarning: The 'labels' parameter of boxplo

OK -> Excel: resumen_sesiones_rodalert_TFM.xlsx
OK -> Figuras en: figuras_rodalert/
OK -> Reporte TFM: resumen_TFM_rodalert.md
